In [ ]:
# Step 1: Import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Load IMDB dataset
# Keras provides IMDB dataset with reviews already tokenized into integers.
# num_words=10000 → keep only the 10,000 most frequent words in the dataset.
num_words = 10000
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=num_words)

print("Train shape:", x_train.shape, "Test shape:", x_test.shape)
print("Example review (first 20 tokens):", x_train[0][:20])
print("Label (1=positive, 0=negative):", y_train[0])

# Step 3: Pad sequences
# Reviews have different lengths → pad/truncate to fixed size (200 words).
# pad_sequences ensures all sequences are the same length → needed for LSTMs.
maxlen = 200
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

print("Padded review shape:", x_train.shape)

# Step 4: Build the LSTM model
# - Embedding: maps word IDs → dense vectors (128 dimensions)
# - LSTM: recurrent network with 64 memory units
# - Dense: final classification layer with sigmoid → probability between 0 and 1
model = keras.Sequential([
    layers.Embedding(input_dim=num_words, output_dim=128, input_length=maxlen),
    layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation="sigmoid")
])

# Compile model
# - Loss: binary_crossentropy (because output is 0/1)
# - Optimizer: Adam (common default optimizer)
# - Metrics: track accuracy
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model.summary())

# Step 5: Train model
# validation_split=0.2 → hold out 20% of training data for validation
# batch_size=64 → train in mini-batches of 64 samples
history = model.fit(
    x_train, y_train,
    epochs=3,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

# Step 6: Evaluate on test data
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}, Loss: {loss:.4f}")

# Step 7: Plot training vs validation accuracy
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.show()

# Step 8: Example prediction
# Pick one test review, check model prediction
sample_idx = 0
sample_review = x_test[sample_idx].reshape(1, -1)
prediction = model.predict(sample_review)[0][0]

print("True label:", y_test[sample_idx])
print("Predicted probability (positive):", prediction)
